In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome import service
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import pandas as pd
from re import findall as re_findall
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
def driver_selenium(typ, url):
    try:
        if typ == 'Chrome':
            driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe')
        elif typ == 'Firefox':
            driver = webdriver.Firefox(executable_path='../driver/geckodriver.exe')
        elif typ == 'Edge':
            driver = webdriver.Edge(executable_path='../driver/msedgedriver.exe')
        driver.get(url)
        return driver
    except:
        print('Loading driver fail')

In [22]:
def indeed_scrap(job_title, location, typ):
    driver = driver_selenium(typ, 'https://indeed.com')
    driver.implicitly_wait(10)
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="text-input-what"]').clear()
    driver.find_element(By.XPATH, '//*[@id="text-input-what"]').send_keys(job_title)
    driver.find_element(By.XPATH, '//*[@id="text-input-where"]').clear()
    driver.find_element(By.XPATH, '//*[@id="text-input-where"]').send_keys(location)
    button = driver.find_element(By.XPATH, '//*[@id="jobsearch"]/button')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(1)
    
    # Button to close google registry
    button_close_google = driver.find_element(By.XPATH, '//*[@id="google-Only-Modal"]/div/div[1]/button')
    driver.execute_script("arguments[0].click();", button_close_google)
    time.sleep(1)
    
    # Cookie close
    button_close_cookie = driver.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]')
    driver.execute_script("arguments[0].click();", button_close_cookie)
    time.sleep(1)
    
    # Droplist time 14 days
#     button_droplist = driver.find_element(By.XPATH, '//*[@id="filter-dateposted"]')
#     driver.execute_script("arguments[0].click();", button_droplist)
    
#     time.sleep(2)
    
#     button_14_day = driver.find_element(By.XPATH, '//*[@id="filter-dateposted-menu"]/li[4]/a')
#     driver.execute_script("arguments[0].click();", button_14_day)
    
#     time.sleep(2)
    
#     # Close another window active
#     button_close_another = driver.find_element(By.XPATH, '//*[@id="popover-x"]/button')
#     driver.execute_script("arguments[0].click();", button_close_another)
#     time.sleep(2)
    return driver

In [4]:
def load_one_page(job_title, location, typ, driver):
    lst_job = []
    #elements = driver.find_elements(By.XPATH, '//*[@id="mosaic-provider-jobcards"]/ul/li')
    #elements = driver.find_elements(By.CLASS_NAME, 'slider_container.css-g7s71f.eu4oa1w0')
    elements = driver.find_elements(By.CLASS_NAME, 'slider_container.css-g7s71f.eu4oa1w0') # stable
    #print(elements)
    for i in range(len(elements)):
        try:
            #driver.execute_script("arguments[0].click();", elements[i])
            #time.sleep(2)
            elements[i].click()
            time.sleep(4)
            one_job = elements[i]
            job_general_html = one_job.get_attribute('outerHTML')
            soup_general = BeautifulSoup(job_general_html, 'lxml')
            
            job_title = soup_general.select('td span')[0].text
            company_name = soup_general.select('td span.companyName')[0].text
            try:
                rating = soup_general.find(class_="ratingNumber").text
            except:
                rating = None
            date_post = soup_general.find(class_="date").text
            date_retrieved = date.today()
            location = soup_general.find(class_='companyLocation').text
            link = 'https://de.indeed.com'+soup_general.find_all('a', href=True)[0]['href']
            
            # Job description. Need to switch to iFrame which is invisible in this case
            
            driver.switch_to.frame("vjs-container-iframe") # Switch to iframe with id name
            job_description = driver.find_element(By.XPATH, '//*[@id="jobDescriptionText"]')
            job_outer_html = job_description.get_attribute('outerHTML')
            soup = BeautifulSoup(job_outer_html, 'lxml')
            job_description_text = soup.text        

            driver.switch_to.default_content() # switch back to the content
            
            job = [job_title, company_name, location, rating, job_description_text, link, date_post, date_retrieved]
            
            lst_job.append(job)            
        except:
            print(f'Element {i} is not loaded properly')
            try:
                button_close_another = driver.find_element(By.XPATH, '//*[@id="popover-x"]/button')
                driver.execute_script("arguments[0].click();", button_close_another)
                time.sleep(2)
            except:
                print('Could not close extension page')
    return lst_job

In [7]:
def load_full(job_title, location, typ):
    driver = indeed_scrap(job_title, location, typ)
    driver.implicitly_wait(10)
    check_next_button = True
    old_click = driver.find_elements(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li')[-1]
    i = 1
    # //*[@id="resultsCol"]/nav/div/ul
    check_last_page = ''
    lst_full = []
    while (i<30) & (check_last_page == ''): # Restrict to maximal 30 pages

        # next_click = driver.find_element(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li[6]/a/span')#.click()
        # driver.execute_script("arguments[0].click();", next_click)  # Javascript execute
        
        lst_one_page = load_one_page(job_title, location, typ, driver)
        lst_full.append(lst_one_page)
        
        try:
            next_click = driver.find_elements(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li')[-1]#.click()        
            next_html = next_click.get_attribute('outerHTML')
            soup = BeautifulSoup(next_html, 'lxml')
            check_last_page = soup.text

            next_click.click()
            print(i)
            i+=1
            time.sleep(5)
        except:
            print(f'Page {i} is not loaded properly')
            driver.execute_script("window.history.go(-2)")
            time.sleep(5)
            #return (lst_full, driver)
    return (lst_full, driver)    

In [14]:
job_title = 'Data Scientist'
location = 'Germany'
typ = 'Chrome'
url = 'https://indeed.com'

In [24]:
driver = indeed_scrap(job_title, location, typ)

C:\Users\Pham Thi Ngan\AppData\Local\Temp\ipykernel_5732\3618989861.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe')


In [26]:
old_click = driver.find_elements(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li')[-1]
i = 1
    # //*[@id="resultsCol"]/nav/div/ul
check_last_page = ''
lst_full = []
while (i<30) & (check_last_page == ''): # Restrict to maximal 30 pages

    # next_click = driver.find_element(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li[6]/a/span')#.click()
    # driver.execute_script("arguments[0].click();", next_click)  # Javascript execute

    lst_one_page = load_one_page(job_title, location, typ, driver)
    lst_full.append(lst_one_page)

    try:
        next_click = driver.find_elements(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li')[-1]#.click()        
        next_html = next_click.get_attribute('outerHTML')
        soup = BeautifulSoup(next_html, 'lxml')
        check_last_page = soup.text

        next_click.click()
        print(i)
        i+=1
        time.sleep(5)
    except:
        print(f'Page {i} is not loaded properly')
        driver.execute_script("window.history.go(-2)")
        time.sleep(5)
        #return (lst_full, driver)

Element 3 is not loaded properly
Element 4 is not loaded properly
Element 5 is not loaded properly
Element 6 is not loaded properly
Element 7 is not loaded properly
Element 8 is not loaded properly
Element 9 is not loaded properly
Element 10 is not loaded properly
Element 11 is not loaded properly
Element 12 is not loaded properly
Element 13 is not loaded properly
Element 14 is not loaded properly
Page 1 is not loaded properly


WebDriverException: Message: chrome not reachable
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A06463+2188387]
	Ordinal0 [0x0099E461+1762401]
	Ordinal0 [0x008B3C40+801856]
	Ordinal0 [0x008A8932+756018]
	Ordinal0 [0x008A9168+758120]
	Ordinal0 [0x008AAA22+764450]
	Ordinal0 [0x008A4379+738169]
	Ordinal0 [0x008B50D0+807120]
	Ordinal0 [0x0090C402+1164290]
	Ordinal0 [0x008FC5F6+1099254]
	Ordinal0 [0x008D6BE0+945120]
	Ordinal0 [0x008D7AD6+948950]
	GetHandleVerifier [0x00CA71F2+2712546]
	GetHandleVerifier [0x00C9886D+2652765]
	GetHandleVerifier [0x00A9002A+520730]
	GetHandleVerifier [0x00A8EE06+516086]
	Ordinal0 [0x009A468B+1787531]
	Ordinal0 [0x009A8E88+1805960]
	Ordinal0 [0x009A8F75+1806197]
	Ordinal0 [0x009B1DF1+1842673]
	BaseThreadInitThunk [0x76D5FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77907A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77907A6E+238]


In [27]:
(lst_job, driver) = load_full(job_title, location, typ)

C:\Users\Pham Thi Ngan\AppData\Local\Temp\ipykernel_5732\3618989861.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe')


1
Element 0 is not loaded properly
Element 1 is not loaded properly
Element 2 is not loaded properly
Element 3 is not loaded properly
Element 4 is not loaded properly
Element 5 is not loaded properly
Element 6 is not loaded properly
Element 7 is not loaded properly
Element 8 is not loaded properly
Element 9 is not loaded properly
Element 10 is not loaded properly
Element 11 is not loaded properly
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Element 3 is not loaded properly
Element 4 is not loaded properly
Element 5 is not loaded properly
Element 6 is not loaded properly
Element 7 is not loaded properly
21
22
23
24
25
26
27
28
29


In [10]:
len(lst_job)

10

In [86]:
driver.implicitly_wait(10)
check_next_button = True
old_click = driver.find_elements(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li')[-1]
i = 1
check_last_page = ''
lst_full = []
while (i<30) & (check_last_page == ''): # Restrict to maximal 30 pages

    # next_click = driver.find_element(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li[6]/a/span')#.click()
    # driver.execute_script("arguments[0].click();", next_click)  # Javascript execute

    lst_one_page = load_one_page(job_title, location, typ, driver)
    lst_full.append(lst_one_page)

    try:
        next_click = driver.find_elements(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li')[-1]#.click()        
        next_html = next_click.get_attribute('outerHTML')
        soup = BeautifulSoup(next_html, 'lxml')
        check_last_page = soup.text

        next_click.click()
        print(i)
        i+=1
        time.sleep(4)
    except:
        print(f'Page {i} is not loaded properly')
        driver.execute_script("window.history.go(-2)")
        time.sleep(5)
        #return (lst_full, driver)

Element 5 is not loaded properly
Element 6 is not loaded properly
Element 7 is not loaded properly
Element 8 is not loaded properly
Element 9 is not loaded properly
Element 10 is not loaded properly
Element 11 is not loaded properly
Element 12 is not loaded properly
Element 13 is not loaded properly
Element 14 is not loaded properly
1
2


In [88]:
len(lst_full)

2

In [69]:
lst_job

[[['Microsoft Business Intelligence Consultant (all genders)',
   'Accenture',
   'Kronberg+1 Ort',
   '4,0',
   '\nWer Unregelmäßigkeiten in der künstlichen Beatmung schwerkranker Kinder frühzeitig erkennt? Wer Zeitungsartikel automatisiert erstellt und Maschinenschäden bis zu 46 Tage im Voraus identifiziert? Die Lösungen von Trivadis mit Hauptsitz in Glattbrugg, Schweiz! Denn Trivadis macht aus Daten Chancen – und aus Chancen Zukunft. Und das macht sich gut im Data & AI Team von Accenture. Seit 2021 ist der IT-Dienstleister mit 15 Standorten in 4 Ländern Teil der globalen Accenture-Familie und stärkt mit seinen rund 750 klugen und lieben Menschen die Data-Science- und AI-Kompetenz des führenden Beratungs- und Technologieunternehmens. Trivadis steht für die hochautomatisierte Bereitstellung und innovative Nutzung von Daten. Und für einige der fähigsten Köpfe der IT-Szene. Du willst rein in die Szene? Dann los!\n Das kannst du erwarten\n\n Als Teammitglied im Experten-Netzwerk lernst d

In [11]:
lst_job_data = []
for i in range(len(lst_job)):
    lst_job_data +=lst_job[i]

In [12]:
len(lst_job_data)

150

In [13]:
len(lst_job_data)

158

In [81]:
lst_job_data

[['Microsoft Business Intelligence Consultant (all genders)',
  'Accenture',
  'Kronberg+1 Ort',
  '4,0',
  '\nWer Unregelmäßigkeiten in der künstlichen Beatmung schwerkranker Kinder frühzeitig erkennt? Wer Zeitungsartikel automatisiert erstellt und Maschinenschäden bis zu 46 Tage im Voraus identifiziert? Die Lösungen von Trivadis mit Hauptsitz in Glattbrugg, Schweiz! Denn Trivadis macht aus Daten Chancen – und aus Chancen Zukunft. Und das macht sich gut im Data & AI Team von Accenture. Seit 2021 ist der IT-Dienstleister mit 15 Standorten in 4 Ländern Teil der globalen Accenture-Familie und stärkt mit seinen rund 750 klugen und lieben Menschen die Data-Science- und AI-Kompetenz des führenden Beratungs- und Technologieunternehmens. Trivadis steht für die hochautomatisierte Bereitstellung und innovative Nutzung von Daten. Und für einige der fähigsten Köpfe der IT-Szene. Du willst rein in die Szene? Dann los!\n Das kannst du erwarten\n\n Als Teammitglied im Experten-Netzwerk lernst du stä

In [14]:
df_job = pd.DataFrame(data=lst_job_data, columns=['position', 'company_name', 'location', 'rating', 'job_description', 'link', 'date_post', 'date_retrived'])

In [15]:
df_job.drop_duplicates(inplace=True)

In [16]:
df_job

,position,company_name,location,rating,job_description,link,date_post,date_retrived
0,Data Scientist (m/f/d),heycar,Berlin,"3,3",\nWe are heycar.\n heycar wants to change the ...,https://de.indeed.com/rc/clk?jk=059f2c17819e15...,Postedvor 4 Tagen,2022-07-22
1,Data Scientist,Digital Associates GmbH,Homeoffice,None,\nWe are currently looking for a Data Scientis...,https://de.indeed.com/rc/clk?jk=68b74873392b6d...,Postedvor 3 Tagen,2022-07-22
2,Data Scientist (f/m/d),Pair Finance GmbH,10623 Berlin,None,Are you ready to change the market?You are a p...,https://de.indeed.com/company/PAIR-Finance-Gmb...,Postedvor 1 Tag,2022-07-22
3,Data Scientist (f/m/d),VineForecast GmbH,Homeoffice,None,Mission \n\n VineForecast is an early-stage st...,https://de.indeed.com/rc/clk?jk=ff1357a23952bc...,PostedHeute,2022-07-22
4,Data Scientist (f/m/d),CoachHub,Berlin,"4,7",\n\n\n\nAbout the job\n Data is important to u...,https://de.indeed.com/rc/clk?jk=91e05cac910b80...,Postedvor 8 Tagen,2022-07-22
...,...,...,...,...,...,...,...,...
153,"Life Scientist - In Vivo Pharmacology, Studies...",Immatics Biotechnologies GmbH,72076 Tübingen+1 Ort,None,\nOverview\nWe are currently seeking full-time...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 1 Tag,2022-07-22
154,Research Assistant/Associate (f/m/d),RWTH Aachen,52074 Aachen,"4,1","\nPhD position for ""Research and Development i...",https://de.indeed.com/rc/clk?jk=c107d50e83ca28...,Postedvor 3 Tagen,2022-07-22
155,Lead Data Scientist,3dprint.com,Berlin,None,\n\n As our Lead Data Scientist you are respo...,https://de.indeed.com/rc/clk?jk=5e6bbcc8a4911c...,Postedvor 7 Tagen,2022-07-22
156,Trainee Data Science in der Unternehmensberatung,niologic,Homeoffice,None,START YOUR CAREER \nWITH OUR R&D AND CONSULTIN...,https://de.indeed.com/rc/clk?jk=8e46e5567b35c8...,PostedHeute,2022-07-22


In [95]:
df_job

,position,company_name,location,rating,job_description,link,date_post,date_retrived
0,Microsoft Business Intelligence Consultant (al...,Accenture,Kronberg+1 Ort,"4,0",\nWer Unregelmäßigkeiten in der künstlichen Be...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 2 Tagen,2022-07-21
1,Business Analyst (m/f/d) - Shipment Routing En...,Hapag Lloyd AG,20095 Hamburg,None,\nWhat is the fastest way to route a shipment ...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 14 Tagen,2022-07-21
2,Director of Business Intelligence & Analytics ...,Usercentrics GmbH,Homeoffice,"5,0",\nDirector of Business Intelligence & Analytic...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 14 Tagen,2022-07-21
3,SAP Berater (m/w/d) – Business Intelligence,akBIT Business & IT Service GbR,Homeoffice in Bad Mergentheim,None,Für unseren Kunden suchen wir aktuell einen SA...,https://de.indeed.comhttps://de.indeed.com/pag...,PostedHeute,2022-07-21
4,Inhouse Consultant (m/w/d) Business Intelligence,techculture GmbH,Homeoffice in Landshut,"5,0",Unser Kunde ist ein weltweit tätiger Werkzeugh...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 1 Tag,2022-07-21
...,...,...,...,...,...,...,...,...
451,Teamleiter:in Data Science Remote - Business I...,Campusjäger by Workwise,Homeoffice in 45127 Essen,"4,3",\nWir sind auf der Suche nach einem Team Lead ...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 9 Tagen,2022-07-21
452,Teamleiter:in Data Science Remote - Business I...,Campusjäger by Workwise,Homeoffice in 50667 Köln,"4,3",\nWir sind auf der Suche nach einem Team Lead ...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 9 Tagen,2022-07-21
453,Teamleiter:in Data Science Remote - Business I...,Campusjäger by Workwise,Homeoffice in 52062 Aachen,"4,3",\nWir sind auf der Suche nach einem Team Lead ...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 9 Tagen,2022-07-21
454,Teamleiter:in Data Science Remote - Business I...,Campusjäger by Workwise,Homeoffice in 68161 Mannheim,"4,3",\nWir sind auf der Suche nach einem Team Lead ...,https://de.indeed.comhttps://de.indeed.com/pag...,Postedvor 9 Tagen,2022-07-21


In [96]:
df_job.to_csv('indeed_business_intelligence_21_07.csv', index=False)